In [7]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import * 
from clapton.depolarization import GateGeneralDepolarizationModel
from qiskit_research.utils.convenience import *
import stim
import numpy as np

np.random.seed(0)

In [8]:
def circuit_to_tableau(circuit: stim.Circuit) -> stim.Tableau:
    s = stim.TableauSimulator()
    s.do_circuit(circuit)
    return s.current_inverse_tableau() ** -1

In [9]:
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))
coeffs

array([0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ,
       0.64589411])

## Altering the Ansatz itself

In [10]:
# let's add a noise model where we specify global 1q and 2q gate errors

# nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.1)
# nm = GateGeneralDepolarizationModel(p2=0.1) #only on 2 qubit gates
nm = None

pauli_twirl = False

# assert not pauli_twirl or nm is not None, "Depolarization model must be defined if Pauli Twirling is applied"

if pauli_twirl:

    init_ansatz = circular_ansatz_mirrored(N=len(paulis[0]), reps=1, fix_2q=True)

    vqe_pcirc = init_ansatz.add_pauli_twirl()
    pauli_twirl_list = [init_ansatz.add_pauli_twirl() for _ in range(100)]

    vqe_pcirc = vqe_pcirc.add_coherent_noise()
    pauli_twirl_list =[circ.add_coherent_noise() for circ in pauli_twirl_list] 
    
    vqe_pcirc.add_pauli_twirl_list(pauli_twirl_list)

    # vqe_pcirc.add_depolarization_model(nm)        
    # pauli_twirl_list = [circuit.add_depolarization_model(nm) for circuit in pauli_twirl_list]

else: 
    vqe_pcirc = circular_ansatz_mirrored(N=len(paulis[0]), reps=1, fix_2q=True)
    
    # vqe_pcirc.add_depolarization_model(nm)
    vqe_pcirc = vqe_pcirc.add_coherent_noise()


In [11]:
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-S-@-S---I-I-------@-S-X-S-I-I-
         |   |               |   |
q1: -I-I-|---X-S-@-S-I-I-@-S-X-S-|---I-I-
         |       |       |       |
q2: -I-I-@-S-----X-S-I-I-X-S-----@-S-I-I-

In [13]:
# pauli_twirl_list[0].stim_circuit().diagram()

In [14]:
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20,                        # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs


/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")



started GA at id None with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


started GA at id 3 with 1 procs


/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0], dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0], dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0], dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0], dtype=object)]
[1, array([0., 0., 0., 0.]), array([3, 3, 1, 3, 1, 3, 3, 0, 3, 0, 1, 0, 1, 2, 0, 1, 1, 2], dtype=object)]
[1, array([0., 0., 0., 0.]), array([3, 3, 1, 3, 1, 3, 3, 0, 3, 0, 1, 0, 1, 2, 0, 1, 1, 2], dtype=object)]
[1, array([0., 0., 0., 0.]), array([3, 3, 1, 3, 1, 3, 3, 0, 3, 0, 1, 0, 1, 2, 0, 1, 1, 2], dtype=object)]
[2, array([0., 0., 0., 0.]), array([3, 3, 1, 3, 3, 3, 3, 0, 3, 0, 1, 0, 1, 2, 0, 1, 1, 2], dtype=object)]
[1, array([0., 0., 0., 0.]), array([3, 3, 1, 3, 1, 3, 3, 0, 3, 0, 1, 0, 1, 2, 0, 1, 1, 2], dtype=object)]
[2, array([0., 0., 0., 0.]), array([3, 3, 1, 3

In [15]:
ks_best

[3, 1, 3, 3, 2, 3, 1, 1, 2, 0, 1, 2, 3, 0, 2, 2, 3, 2]

In [16]:
energy_noisy, energy_noiseless

(0.0, 0.0)

In [17]:
# differrence
np.abs(energy_noisy-energy_noiseless)

0.0

In [ ]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -Y------SQRT_Y_DAG-X-S-@-S---I-SQRT_Y_DAG-------@-S-X-S-Y-SQRT_Y-
                       |   |                        |   |
q1: -Y------SQRT_Y-----|---X-S-@-S-SQRT_Y_DAG-I-@-S-X-S-|---I-SQRT_Y-
                       |       |                |       |
q2: -SQRT_Y-SQRT_Y_DAG-@-S-----X-S-I----------Y-X-S-----@-S-Y-Y------